In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join('.','mnist'), one_hot=True)
X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [3]:
tf.reset_default_graph()

In [4]:
n_classes=10
n_width=28
n_height=28
n_depth=1
n_inputs=n_height*n_width*n_depth
learning_rate=0.001
n_epochs=10
batch_size=100
n_batches=int(mnist.train.num_examples/batch_size)
    
    #input images shape: (n_samples,n_pixels)

x = tf.placeholder(dtype=tf.float32, name="x", shape=[None, n_inputs]) 
    #output labels
y=tf.placeholder(dtype=tf.float32,name='y',shape=[None,n_classes])

# reshape input to (n_samples,n_width,n_height,n_depth)
x_ = tf.reshape(x, shape=[-1, n_width, n_height, n_depth])

In [5]:
#layer 1
layer1_w=tf.Variable(tf.random_normal(shape=[4,4,n_depth,32],stddev=0.1),name='l1w')
layer1_b=tf.Variable(tf.random_normal([32]),name='l1b')

layer1_conv=tf.nn.relu(tf.nn.conv2d(x_,layer1_w,strides=[1,1,1,1],padding='SAME')+layer1_b)

#pool 
layer1_pool=tf.nn.max_pool(layer1_conv,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [6]:
#layer 2
layer2_w=tf.Variable(tf.random_normal(shape=[4,4,32,64],stddev=0.1),name='l2w')
layer2_b=tf.Variable(tf.random_normal([64]),name='l2b')

layer2_conv=tf.nn.relu(tf.nn.conv2d(layer1_pool,layer2_w,strides=[1,1,1,1],padding='SAME')+layer2_b)

#pool 
layer2_pool=tf.nn.max_pool(layer2_conv,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [7]:
#layer 3
layer3_w=tf.Variable(tf.random_normal(shape=[64*7*7*1,1024],stddev=0.01),name='l3_w')
layer3_b=tf.Variable(tf.random_normal([1024]),name='l3_b')

#fully connected
layer3_fc=tf.nn.relu(tf.matmul(tf.reshape(layer2_pool,[-1,64*7*7*1]),layer3_w)+layer3_b)

In [8]:
# layer 4
layer4_w=tf.Variable(tf.random_normal(shape=[1024,n_classes],stddev=0.01),name='l4_w')
layer4_b=tf.Variable(tf.random_normal(shape=[n_classes]),name='l4_b')

layer4_out=tf.matmul(layer3_fc,layer4_w)+layer4_b


In [9]:
model=layer4_out

In [10]:
entropy=tf.nn.softmax_cross_entropy_with_logits(logits=model,labels=y)
loss=tf.reduce_mean(entropy)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
# train and evaluate
with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        total_loss = 0.0
        for batch in range(n_batches):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            feed_dict={x:batch_x, y: batch_y}
            batch_loss,_ = tfs.run([loss, optimizer],
                                   feed_dict=feed_dict
                                  )
            total_loss += batch_loss 
        average_loss = total_loss / n_batches
        print("Epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))
    print("Model Trained.")

    predictions_check = tf.equal(tf.argmax(model,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(predictions_check, tf.float32))
    feed_dict = {x:mnist.test.images, y:mnist.test.labels}
    print("Accuracy:", accuracy.eval(feed_dict=feed_dict))

